<div align="center">

# Использование k-кратной перекрестной проверки для оценки производительности модели

</div>

---


### Метод откладывания (Holdout method)

* **Суть метода**:
  Делим исходный набор данных на два поднабора:

  * **Обучающий** — для построения модели.
  * **Тестовый** — для оценки её обобщающей способности на новых данных.


#### Ограничения и проблемы

* **Утечка тестовых данных (data leakage)**:
  Если в процессе выбора модели и настройки параметров многократно использовать один и тот же тестовый набор, модель "запоминает" этот набор, и тест перестает быть независимым. Это приводит к **переобучению** и завышенной оценке качества.

* Несмотря на это, многие специалисты продолжают использовать тестовые данные для выбора модели — **это плохая практика**.


#### Более надежный подход: разделение на три поднабора

* **Обучающий набор (training)** — обучение моделей с разными гиперпараметрами.
* **Валидационный набор (validation)** — оценка моделей, выбор лучшей по результатам.
* **Тестовый набор (test)** — финальная независимая проверка производительности выбранной модели.

**Преимущество**: тестовый набор остается полностью изолированным от этапов обучения и выбора, что обеспечивает **надежную и непредвзятую оценку**.


#### Недостатки метода откладывания

* Результаты оценки сильно зависят от конкретного разбиения данных на обучающую и тестовую выборки.
* Это приводит к **высокой вариативности оценок**: разные разбиения могут дать сильно отличающиеся оценки качества модели.


#### Итог

* Метод откладывания — простой и популярный, но с ограничениями.
* Для более объективной оценки и настройки моделей лучше использовать **три поднабора** (train/validation/test) или более продвинутые методы, например, перекрестную проверку (cross-validation).

---


### K-кратная перекрестная проверка (Cross-validation)

* **Идея**: обучающий набор данных случайно разбивают на k непересекающихся подвыборок (folds).
* **Процесс**:

  * k раз: берут k-1 подвыборок для обучения модели, 1 подвыборку — для теста.
  * Получают k моделей и k оценок производительности.
* **Средняя оценка**: средняя по всем k тестам даёт более устойчивую оценку, менее зависимую от конкретного разбиения, чем простой метод откладывания.


#### Применение и цели

* Используется для **настройки гиперпараметров** — подбор оптимальных значений, которые обеспечивают хорошую обобщающую способность модели.
* После выбора параметров модель обучают заново на полном наборе данных для получения **окончательной модели** с более точной оценкой на независимых тестах.


#### Особенности и преимущества

* Каждый объект данных используется **ровно один раз в тесте** и **k-1 раз в обучении** (без пересечений).
* Тестовые подвыборки не пересекаются друг с другом.
* Используются все данные, что делает оценку производительности более надежной.
* Обычно берут **k=10** — оптимальный баланс между смещением и разбросом оценок (на основании исследований Рона Кохави).



#### Вариации и рекомендации

* При **малых объемах данных** можно увеличить k, чтобы получить более точные оценки (но возрастает вычислительная нагрузка и дисперсия оценок).
* При **больших данных** можно брать меньшее k (например, k=5), чтобы уменьшить время обучения и оценки без потери точности.
* **Leave-One-Out CV (LOOCV)** — частный случай k-кратной с k равным количеству записей, тестируется по одному примеру; подходит для очень маленьких наборов.
* **Стратифицированная перекрестная проверка** — гарантирует, что доля классов в каждом fold совпадает с долей в полном наборе, улучшая оценку для несбалансированных классов.

---



In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [36]:
# Загрузка набора данных из локального хранилища
df = pd.read_csv('~/Рабочий стол/ML/Data/wdbc.data',
                 header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [37]:
# Разделяем данные на
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# Преобразуем метки классов (диагнозы) в целые числа
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

# Проверка сопоставлений le
le.transform(['M', 'B'])

# Разделение на train и test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.20,
                     stratify = y,
                     random_state = 1)

# Объединение всех этапов в конвейер
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression())

pipe_lr.fit(X_train, y_train)

,steps,"[('standardscaler', ...), ('pca', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_components,2
,copy,True
,whiten,False
,svd_solver,'auto'


In [38]:
# Стратифицированная проверка
kfold = StratifiedKFold(n_splits = 10).split(X_train, y_train)
scores = cross_val_score(estimator = pipe_lr,
                         X = X_train,
                         y = y_train,
                         cv = 10, # кол-во выборок
                         n_jobs = 1) # Кол-во ядер
print(f'Оценки точности по CV: {scores}')

Оценки точности по CV: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]


In [39]:
print(f'Точность по CV: {np.mean(scores):.3f} '
      f'+/- {np.std(scores):.3f}')

Точность по CV: 0.950 +/- 0.014


### Вывод о k-кратной перекрестной проверке

* **k-кратная перекрестная проверка** — это мощный и надежный метод оценки качества модели машинного обучения, который повторно разбивает обучающие данные на k непересекающихся подвыборок (folds), обучая модель k раз и тестируя на каждом отдельном fold.

* В нашем примере используется **стратифицированная 10-кратная перекрестная проверка (StratifiedKFold с k=10)**, что обеспечивает сохранение пропорций классов в каждой подвыборке и повышает качество оценки при наличии несбалансированных классов.

* Конвейер (pipeline) объединяет важные этапы предобработки: стандартизацию признаков, понижение размерности с помощью PCA и обучение логистической регрессии. Это гарантирует корректное применение всех преобразований внутри каждого fold без утечки информации между обучающими и тестовыми подвыборками.

* Результат:

  * В каждой из 10 итераций получаем точность модели на отложенной подвыборке.
  * Среднее значение точности (`np.mean(scores)`) и стандартное отклонение (`np.std(scores)`) показывают как среднюю производительность, так и вариативность оценки.

* **Преимущества данного подхода:**

  * Использование всех данных для обучения и тестирования (каждая точка данных попадает в тест ровно один раз).
  * Более стабильная и менее смещённая оценка, чем при простом разбиении на train/test.
  * Стратификация сохраняет баланс классов, что особенно важно при диагностике заболеваний (как в твоем наборе данных).

* **Практическое значение:**

  * Средняя точность по CV — объективная метрика качества модели.
  * Стандартное отклонение помогает понять, насколько надежна оценка, и есть ли сильная зависимость результатов от конкретных разбиений данных.

---

**k-кратная перекрестная проверка — это лучший способ оценить качество модели и её способность к обобщению, особенно при ограниченных данных и необходимости настроить гиперпараметры. Стратификация добавляет надежности оценкам в задачах с несбалансированными классами.**
